## Import required packages

In [0]:
# Download required packages
!pip -q install gdown missingno torch

%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf
from pyspark.sql.window import Window

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
import torch
import torch.nn as nn
from typing import *
import datetime
import gdown

import tqdm as tq
def tqdm(*args, **kwargs):
  ''' Small trick to prevent tqdm printing newlines at each step. '''
  return tq.tqdm(*args, **kwargs, leave=True, position=0)

WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

## Data aquisition
We retrieve our datasets and download them to a temporary directory in the driver node.

In [0]:
gdown.download('https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM', '/tmp/data.zip', quiet=False)
!unzip -q /tmp/data.zip -d /tmp/
!rm /tmp/data.zip

Downloading...
From: https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM
To: /tmp/data.zip
0.00B [00:00, ?B/s]1.57MB [00:00, 14.4MB/s]4.72MB [00:00, 22.8MB/s]8.91MB [00:00, 30.4MB/s]15.7MB [00:00, 43.8MB/s]20.4MB [00:00, 41.8MB/s]25.7MB [00:00, 39.5MB/s]34.1MB [00:00, 33.3MB/s]42.5MB [00:01, 33.1MB/s]59.2MB [00:01, 43.4MB/s]71.9MB [00:01, 45.5MB/s]

Then we load the datasets to the DBFS.

In [0]:
dbutils.fs.mv("file:/tmp/data", "dbfs:/data", recurse=True)

Out[3]: True

In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


## Dataset loading

In [0]:
key_stats_df = spark.read.load("dbfs:/data/key_stats_yahoo.csv", 
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true"
                          )

# Drop the first ID column
key_stats_df = key_stats_df.drop(key_stats_df.columns[0])

# Use legacy format to parse dates
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
key_stats_df = key_stats_df.withColumn("Date", F.to_date(key_stats_df["Date"], 'MM/dd/yyyy HH:mm'))

# Cast numerical columns to double
for column in key_stats_df.columns[2:]:
  key_stats_df = key_stats_df.withColumn(column, key_stats_df[column].cast("double"))

# Prices dataframes for every stock
prices_files = [f.path for f in dbutils.fs.ls('/data/prices/') if f.path.endswith('.csv')]
dfs_names = [f.rsplit('/', 1)[1][:-len('.csv')] for f in prices_files]
prices_dfs = []
for f in tqdm(prices_files, desc='Reading stock price data', total=len(prices_files)):
  df = spark.read.load(f,
                       format="csv",
                       sep=",",
                       inferSchema="true",
                       header="true"
                      )
  df = df.withColumn("Date", F.to_date(df["Date"], 'dd-MM-yyyy'))
  prices_dfs.append(df)

Reading stock price data: 0%| | 0/429 [00:00<?, ?it/s]Reading stock price data: 0%| | 1/429 [00:02<17:12, 2.41s/it]Reading stock price data: 0%| | 2/429 [00:03<12:46, 1.80s/it]Reading stock price data: 1%| | 3/429 [00:04<10:52, 1.53s/it]Reading stock price data: 1%| | 4/429 [00:06<10:06, 1.43s/it]Reading stock price data: 1%| | 5/429 [00:07<09:28, 1.34s/it]Reading stock price data: 1%|▏ | 6/429 [00:08<08:37, 1.22s/it]Reading stock price data: 2%|▏ | 7/429 [00:09<08:37, 1.23s/it]Reading stock price data: 2%|▏ | 8/429 [00:10<08:44, 1.25s/it]Reading stock price data: 2%|▏ | 9/429 [00:12<08:37, 1.23s/it]Reading stock price data: 2%|▏ | 10/429 [00:13<08:26, 1.21s/it]Reading stock price data: 3%|▎ | 11/429 [00:14<08:18, 1.19s/it]Reading stock price data: 3%|▎ | 12/429 [00:15<08:28, 1.22s/it]Reading stock price data: 3%|▎ | 13/429 [00:16<08:03, 1.16s/it]Reading stock price data: 3%|▎ | 14/429 [00:17<07:35, 1.10s/it]Reading stock price data: 3%|▎ | 15/429 [00:18<07:36, 1.10s/it]Reading stock price data: 4%|▎ | 16/429 [00:20<07:51, 1.14s/it]Reading stock price data: 4%|▍ | 17/429 [00:21<07:39, 1.12s/it]Reading stock price data: 4%|▍ | 18/429 [00:23<09:34, 1.40s/it]Reading stock price data: 4%|▍ | 19/429 [00:24<08:37, 1.26s/it]Reading stock price data: 5%|▍ | 20/429 [00:25<08:02, 1.18s/it]Reading stock price data: 5%|▍ | 21/429 [00:26<07:35, 1.12s/it]Reading stock price data: 5%|▌ | 22/429 [00:26<07:00, 1.03s/it]Reading stock price data: 5%|▌ | 23/429 [00:27<06:38, 1.02it/s]Reading stock price data: 6%|▌ | 24/429 [00:28<06:49, 1.01s/it]Reading stock price data: 6%|▌ | 25/429 [00:29<06:56, 1.03s/it]Reading stock price data: 6%|▌ | 26/429 [00:31<07:26, 1.11s/it]Reading stock price data: 6%|▋ | 27/429 [00:32<07:01, 1.05s/it]Reading stock price data: 7%|▋ | 28/429 [00:33<06:57, 1.04s/it]Reading stock price data: 7%|▋ | 29/429 [00:34<06:42, 1.01s/it]Reading stock price data: 7%|▋ | 30/429 [00:35<06:41, 1.01s/it]Reading stock price data: 7%|▋ | 31/429 [00:36<06:28, 1.02it/s]Reading stock price data: 7%|▋ | 32/429 [00:36<06:28, 1.02it/s]Reading stock price data: 8%|▊ | 33/429 [00:37<06:13, 1.06it/s]Reading stock price data: 8%|▊ | 34/429 [00:38<06:17, 1.05it/s]Reading stock price data: 8%|▊ | 35/429 [00:40<06:40, 1.02s/it]Reading stock price data: 8%|▊ | 36/429 [00:41<07:39, 1.17s/it]Reading stock price data: 9%|▊ | 37/429 [00:42<07:23, 1.13s/it]Reading stock price data: 9%|▉ | 38/429 [00:43<07:02, 1.08s/it]Reading stock price data: 9%|▉ | 39/429 [00:44<06:32, 1.01s/it]Reading stock price data: 9%|▉ | 40/429 [00:45<06:09, 1.05it/s]Reading stock price data: 10%|▉ | 41/429 [00:46<06:27, 1.00it/s]Reading stock price data: 10%|▉ | 42/429 [00:47<06:37, 1.03s/it]Reading stock price data: 10%|█ | 43/429 [00:48<06:21, 1.01it/s]Reading stock price data: 10%|█ | 44/429 [00:49<06:06, 1.05it/s]Reading stock price data: 10%|█ | 45/429 [00:50<06:33, 1.03s/it]Reading stock price data: 11%|█ | 46/429 [00:51<06:02, 1.06it/s]Reading stock price data: 11%|█ | 47/429 [00:52<05:59, 1.06it/s]Reading stock price data: 11%|█ | 48/429 [00:52<05:43, 1.11it/s]Reading stock price data: 11%|█▏ | 49/429 [00:53<05:42, 1.11it/s]Reading stock price data: 12%|█▏ | 50/429 [00:54<05:42, 1.11it/s]Reading stock price data: 12%|█▏ | 51/429 [00:55<05:31, 1.14it/s]Reading stock price data: 12%|█▏ | 52/429 [00:56<05:43, 1.10it/s]Reading stock price data: 12%|█▏ | 53/429 [00:57<05:40, 1.11it/s]Reading stock price data: 13%|█▎ | 54/429 [00:58<05:22, 1.16it/s]Reading stock price data: 13%|█▎ | 55/429 [00:59<06:05, 1.02it/s]Reading stock price data: 13%|█▎ | 56/429 [01:00<06:37, 1.07s/it]Reading stock price data: 13%|█▎ | 57/429 [01:01<06:22, 1.03s/it]Reading stock price data: 14%|█▎ | 58/429 [01:02<06:06, 1.01it/s]Reading stock price data: 14%|█▍ | 59/429 [01:03<05:38, 1.09it/s]Reading stock price data: 14%|█▍ | 60/429 [01:04<05:29, 1.12it/s]Reading stock price data: 14%|█▍ | 61/429 [01:04<05:13, 1.17it/s]Reading stock price d

## Dataset analysis

In [0]:
print("Key stats dataframe format:")
prices_dfs[0].printSchema()

Key stats dataframe format:
root
-- Date: date (nullable = true)
-- Low: double (nullable = true)
-- Open: double (nullable = true)
-- Volume: integer (nullable = true)
-- High: double (nullable = true)
-- Close: double (nullable = true)
-- Adjusted Close: double (nullable = true)

In [0]:
print("Prices dataframe format:")
key_stats_df.printSchema()

Prices dataframe format:
root
-- Date: date (nullable = true)
-- Ticker: string (nullable = true)
-- Price: double (nullable = true)
-- DE Ratio: double (nullable = true)
-- Trailing P/E: double (nullable = true)
-- Price/Sales: double (nullable = true)
-- Price/Book: double (nullable = true)
-- Profit Margin: double (nullable = true)
-- Operating Margin: double (nullable = true)
-- Return on Assets: double (nullable = true)
-- Return on Equity: double (nullable = true)
-- Revenue Per Share: double (nullable = true)
-- Market Cap: double (nullable = true)
-- Enterprise Value: double (nullable = true)
-- Forward P/E: double (nullable = true)
-- PEG Ratio: double (nullable = true)
-- Enterprise Value/Revenue: double (nullable = true)
-- Enterprise Value/EBITDA: double (nullable = true)
-- Revenue: double (nullable = true)
-- Gross Profit: double (nullable = true)
-- EBITDA: double (nullable = true)
-- Net Income Avl to Common : double (nullable = true)
-- Diluted EPS: double (nullable = true)
-- Earnings Growth: double (nullable = true)
-- Revenue Growth: double (nullable = true)
-- Total Cash: double (nullable = true)
-- Total Cash Per Share: double (nullable = true)
-- Total Debt: double (nullable = true)
-- Current Ratio: double (nullable = true)
-- Book Value Per Share: double (nullable = true)
-- Cash Flow: double (nullable = true)
-- Beta: double (nullable = true)

### Utility functions

In [0]:
# TODO: add remaining utility functions

def prices_df_nan_summary(prices_dfs: List[pyspark.sql.DataFrame], names: List[str]) -> pd.DataFrame:
  ''' Utility function to summarize columns that have missing values. '''
  nan_dfs = []
  for prices_df, name in tqdm(zip(prices_dfs, names), total=len(prices_dfs)):
    nan_absolute = prices_df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in prices_df.columns]).first()
    if any(nan_absolute):
      # Simple conversion from Pyspark row -> Python set of values
      values = set(nan_absolute.asDict().values()).difference({0})
      # Either we don't have values for that row, or we have all of them (but Date which is non-nullable)
      # Values contains the no. of NaN values and 0 in correspondance of the Date column
      assert len(values) == 1
      nan_count = values.pop()
      nan_dfs.append((name, round(100*nan_count/prices_df.count(), 3), nan_count))

  return pd.DataFrame(nan_dfs, columns=['Stock name', 'Missing data (%)', 'Count'])

def remove_trailing_nan(df: pyspark.sql.DataFrame, ticker: str, col: str = 'Low') -> pyspark.sql.DataFrame:
  df_length = df.count()
  
  df_sorted_id = df.sort('Date').withColumn('id', F.row_number().over(Window.orderBy('Date')))
  
  cumsum_df = df_sorted_id.withColumn('cumsum', F.when(F.isnull(df_sorted_id.Low), F.row_number().over(Window.partitionBy('Low').orderBy('Date'))).otherwise(0))

  end_idx = cumsum_df.where(cumsum_df['id'] == df_length).first().cumsum
  
  return df_sorted_id.where(df_sorted_id['id'] <= df_length-end_idx+1)

def merge_prices_fundamentals(
    prices_dfs: List[pyspark.sql.DataFrame],
    key_stats_df: pyspark.sql.DataFrame,
    dfs_names: List[str],
    drop_cols: List[str] = ['Date', 'Ticker', 'Price']
    ) -> List[pyspark.sql.DataFrame]:
  pass                
                
def fill_missing_days(aggregate_dfs: List[pyspark.sql.DataFrame], remove_weekends: bool = True, end_year: int = 2013) -> List[pyspark.sql.DataFrame]:
  pass

def missing_values_summary(df):
  ''' Returns a utility summary to view missing values in our dataframe. '''
  n = df.count()
  
  def to_percentage(x: pyspark.sql.column.Column, n: int) -> int:
    ''' Utility function to compute the amount of missing values as a percentage of the original dataframe. '''
    return F.round(100 * x / n, 3)
  
  # Aggregate using the count function over null values, and return a view over the obtained (single row) dataframe
  return df.agg(*[to_percentage(F.count(F.when(F.isnull(c), c)), n).alias(c) for c in df.columns]).first()

In [0]:
print("Overview of the missing values in the key_stats dataframe\n")
key_stats_summary = missing_values_summary(key_stats_df)
key_stats_summary

Overview of the missing values in the key_stats dataframe

Out[16]: Row(Date=0.0, Ticker=0.0, Price=3.657, DE Ratio=16.886, Trailing P/E=8.432, Price/Sales=0.548, Price/Book=2.354, Profit Margin=0.712, Operating Margin=1.391, Return on Assets=2.508, Return on Equity=3.011, Revenue Per Share=0.241, Market Cap=0.285, Enterprise Value=0.723, Forward P/E=41.251, PEG Ratio=4.325, Enterprise Value/Revenue=0.942, Enterprise Value/EBITDA=10.633, Revenue=0.909, Gross Profit=3.92, EBITDA=10.37, Net Income Avl to Common =0.055, Diluted EPS=0.296, Earnings Growth=15.966, Revenue Growth=2.146, Total Cash=0.854, Total Cash Per Share=0.712, Total Debt=3.844, Current Ratio=10.37, Book Value Per Share=0.449, Cash Flow=9.801, Beta=4.413)

### Missing values imputation

In [0]:
summary = prices_df_nan_summary(prices_dfs, dfs_names)
px.bar(summary, x='Stock name', y='Missing data (%)', hover_data=['Count'], title="Stock price dataset before preprocessing (only columns with missing values are displayed)")

0%| | 0/429 [00:00<?, ?it/s] 0%| | 1/429 [00:00<04:21, 1.64it/s] 0%| | 2/429 [00:01<03:56, 1.80it/s] 1%| | 3/429 [00:01<03:38, 1.95it/s] 1%| | 4/429 [00:02<04:24, 1.61it/s] 1%| | 5/429 [00:02<03:55, 1.80it/s] 1%|▏ | 6/429 [00:03<03:41, 1.91it/s] 2%|▏ | 7/429 [00:03<03:35, 1.96it/s] 2%|▏ | 8/429 [00:04<03:42, 1.89it/s] 2%|▏ | 9/429 [00:04<03:36, 1.94it/s] 2%|▏ | 10/429 [00:05<03:38, 1.92it/s] 3%|▎ | 11/429 [00:05<03:49, 1.82it/s] 3%|▎ | 12/429 [00:06<04:27, 1.56it/s] 3%|▎ | 13/429 [00:07<04:27, 1.55it/s] 3%|▎ | 14/429 [00:08<04:34, 1.51it/s] 3%|▎ | 15/429 [00:08<04:25, 1.56it/s] 4%|▎ | 16/429 [00:09<04:04, 1.69it/s] 4%|▍ | 17/429 [00:09<03:44, 1.84it/s] 4%|▍ | 18/429 [00:10<03:39, 1.87it/s] 4%|▍ | 19/429 [00:10<03:25, 1.99it/s] 5%|▍ | 20/429 [00:11<03:23, 2.01it/s] 5%|▍ | 21/429 [00:11<03:19, 2.05it/s] 5%|▌ | 22/429 [00:11<03:08, 2.16it/s] 5%|▌ | 23/429 [00:12<03:07, 2.17it/s] 6%|▌ | 24/429 [00:12<03:12, 2.10it/s] 6%|▌ | 25/429 [00:13<03:23, 1.98it/s] 6%|▌ | 26/429 [00:14<03:31, 1.91it/s] 6%|▋ | 27/429 [00:14<03:40, 1.82it/s] 7%|▋ | 28/429 [00:15<04:12, 1.59it/s] 7%|▋ | 29/429 [00:15<03:45, 1.78it/s] 7%|▋ | 30/429 [00:16<03:30, 1.90it/s] 7%|▋ | 31/429 [00:16<03:26, 1.92it/s] 7%|▋ | 32/429 [00:17<03:34, 1.85it/s] 8%|▊ | 33/429 [00:18<03:46, 1.75it/s] 8%|▊ | 34/429 [00:18<04:06, 1.60it/s] 8%|▊ | 35/429 [00:19<04:09, 1.58it/s] 8%|▊ | 36/429 [00:19<03:54, 1.68it/s] 9%|▊ | 37/429 [00:20<03:45, 1.74it/s] 9%|▉ | 38/429 [00:21<03:38, 1.79it/s] 9%|▉ | 39/429 [00:21<03:26, 1.89it/s] 9%|▉ | 40/429 [00:22<03:58, 1.63it/s] 10%|▉ | 41/429 [00:22<03:58, 1.62it/s] 10%|▉ | 42/429 [00:23<04:27, 1.45it/s] 10%|█ | 43/429 [00:24<04:07, 1.56it/s] 10%|█ | 44/429 [00:24<03:38, 1.76it/s] 10%|█ | 45/429 [00:25<03:56, 1.62it/s] 11%|█ | 46/429 [00:26<04:07, 1.55it/s] 11%|█ | 47/429 [00:26<03:58, 1.60it/s] 11%|█ | 48/429 [00:27<03:49, 1.66it/s] 11%|█▏ | 49/429 [00:27<03:37, 1.75it/s] 12%|█▏ | 50/429 [00:28<03:29, 1.81it/s] 12%|█▏ | 51/429 [00:28<03:21, 1.88it/s] 12%|█▏ | 52/429 [00:29<03:11, 1.97it/s] 12%|█▏ | 53/429 [00:29<03:13, 1.94it/s] 13%|█▎ | 54/429 [00:30<03:52, 1.61it/s] 13%|█▎ | 55/429 [00:31<04:05, 1.52it/s] 13%|█▎ | 56/429 [00:32<04:04, 1.53it/s] 13%|█▎ | 57/429 [00:32<03:59, 1.55it/s] 14%|█▎ | 58/429 [00:33<03:53, 1.59it/s] 14%|█▍ | 59/429 [00:33<03:51, 1.60it/s] 14%|█▍ | 60/429 [00:34<03:41, 1.67it/s] 14%|█▍ | 61/429 [00:34<03:26, 1.78it/s] 14%|█▍ | 62/429 [00:35<03:18, 1.85it/s] 15%|█▍ | 63/429 [00:35<03:14, 1.88it/s] 15%|█▍ | 64/429 [00:36<03:18, 1.84it/s] 15%|█▌ | 65/429 [00:37<03:26, 1.76it/s] 15%|█▌ | 66/429 [00:37<03:48, 1.59it/s] 16%|█▌ | 67/429 [00:38<03:38, 1.66it/s] 16%|█▌ | 68/429 [00:38<03:32, 1.70it/s] 16%|█▌ | 69/429 [00:39<03:25, 1.76it/s] 16%|█▋ | 70/429 [00:40<03:21, 1.78it/s] 17%|█▋ | 71/429 [00:40<03:14, 1.84it/s] 17%|█▋ | 72/429 [00:41<03:09, 1.88it/s] 17%|█▋ | 73/429 [00:41<03:40, 1.62it/s] 17%|█▋ | 74/429 [00:42<03:31, 1.68it/s] 17%|█▋ | 75/429 [00:42<03:19, 1.78it/s] 18%|█▊ | 76/429 [00:43<03:09, 1.86it/s] 18%|█▊ | 77/429 [00:44<03:32, 1.66it/s] 18%|█▊ | 78/429 [00:45<04:48, 1.22it/s] 18%|█▊ | 79/429 [00:46<04:29, 1.30it/s] 19%|█▊ | 80/429 [00:46<04:12, 1.38it/s] 19%|█▉ | 81/429 [00:47<04:00, 1.45it/s] 19%|█▉ | 82/429 [00:47<03:53, 1.49it/s] 19%|█▉ | 83/429 [00:48<03:44, 1.54it/s] 20%|█▉ | 84/429 [00:49<03:26, 1.67it/s] 20%|█▉ | 85/429 [00:49<03:18, 1.73it/s] 20%|██ | 86/429 [00:50<03:33, 1.61it/s] 20%|██ | 87/429 [00:51<03:47, 1.51it/s] 21%|██ | 88/429 [00:51<03:59, 1.42it/s] 21%|██ | 89/429 [00:52<03:30, 1.62it/s] 21%|██ | 90/429 [00:52<03:31, 1.60it/s] 21%|██ | 91/429 [00:53<03:19, 1.70it/s] 21%|██▏ | 92/429 [00:54<03:29, 1.61it/s] 22%|██▏ | 93/429 [00:54<03:29, 1.61it/s] 22%|██▏ | 94/429 [00:55<03:19, 1.68it/s] 22%|██▏ | 95/429 [00:55<02:57, 1.88it/s] 22%|██▏ | 96/429 [00:56<02:59, 1.85it/s] 23%|██▎ | 97/429 [00:57<03:56, 1.40it/s] 23%|██▎ | 98/429 [00:58<03:56, 1.40it/s] 23%|██▎ | 99/429 [00:58<03:38, 1.51it/s] 23%|█

For most of the above stocks with missing values, we noticed that they indeed exist up to a given time and after that no more data is available. It may due to a business failure, hence no more stocks will be exchanged from that moment on.

In [0]:
# Clear our input data from training NaN values
prices_dfs_new = [remove_trailing_nan(df,name) for df,name in tqdm(zip(prices_dfs, dfs_names), total=len(prices_dfs))]

# Remove INTH stock from our dataset since it contains many inactivity periods
inth_idx = dfs_names.index('INTH')
del dfs_names[inth_idx]
del prices_dfs_new[inth_idx]

summary = prices_df_nan_summary(prices_dfs_new, dfs_names)
px.bar(summary, x='Stock name', y='Missing data (%)', hover_data=['Count'], title="Stock price dataset after preprocessing (only columns with missing values are displayed)")

Out[199]:

At this point we use the fast forward imputation technique to fill-in missing values. Please note that in this case missing values are mostly due to holidays or periods when stocks are not exchanged.

### Building our new dataset